In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from bokeh.io import show, output_notebook
output_notebook()

from scripts_viz.visualization_utils import *
from scripts_viz.visualization_perfgrid import *
from scripts_viz.visualization_utils import TTQcolor
from scripts_mlflow.mlflow_utils import *

from bokeh.layouts import gridplot, row, column

import mlflow


Loading BokehJS ...

In [2]:
pd.set_option("display.max_rows", 101)

In [3]:
expnames = set([exp.name for exp in mlflow.tracking.MlflowClient().list_experiments()])
expnames

{'MLP_enriched_time_seq26000_2000_opt_p90',
 'benchmarks_shuffle_imp',
 'benchmarks_shuffle_p180',
 'benchmarks_shuffle_p90',
 'benchmarks_time_old_imp',
 'benchmarks_time_old_p180',
 'benchmarks_time_old_p90',
 'benchmarks_time_opt_imp',
 'benchmarks_time_opt_p180',
 'benchmarks_time_opt_p90',
 'enriched_shuffle_imp',
 'enriched_shuffle_p180',
 'enriched_shuffle_p90',
 'enriched_time_seq24000_6000_imp',
 'enriched_time_seq24000_6000_opt_imp',
 'enriched_time_seq26000_2000_opt_p180',
 'enriched_time_seq26000_2000_opt_p90',
 'enriched_time_seq26000_2000_p180',
 'enriched_time_seq26000_2000_p90'}

In [4]:
experiment = 'MLP_enriched_time_seq26000_2000_opt_p90'

In [5]:
viz = create_exp_df(experiment)

In [73]:
viz

,ANN_12_190827_1251,ANN_24_190827_1325,ANN_190827_1228
train_size,46101,46101,46101
early_stopping,True,True,True
loss_func,BinaryCrossentropy,BinaryCrossentropy,BinaryCrossentropy
tr_val_shuffle,False,False,False
early_stopping_metric,"('accuracy', 0.9)","('accuracy', 0.9)","('accuracy', 0.9)"
tr_time_str,Training completed in 3 mins and 11.09 secs,Training completed in 3 mins and 11.77 secs,Training completed in 3 mins and 23.49 secs
class_1_weight,25,10,25
model_filename,time_2018-04-30_p90_bg__sequential_12_190827_1...,time_2018-04-30_p90_bg__sequential_24_190827_1...,time_2018-04-30_p90_bg__sequential_190827_1228.h5
pred_threshold,0.55,0.55,0.55
model_filepath,../data/models/MLP_enriched_time_seq26000_2000...,../data/models/MLP_enriched_time_seq26000_2000...,../data/models/MLP_enriched_time_seq26000_2000...


In [72]:
viz.loc['model_filepath']

ANN_12_190827_1251    ../data/models/MLP_enriched_time_seq26000_2000...
ANN_24_190827_1325    ../data/models/MLP_enriched_time_seq26000_2000...
ANN_190827_1228       ../data/models/MLP_enriched_time_seq26000_2000...
Name: model_filepath, dtype: object

In [66]:
hnn = viz.loc['hidden_nodes'].str.strip().str.split('[').apply(lambda x:x[1].split(']')[0].strip().split(','))
hnn = hnn.apply(lambda x:[h for h in x])

In [67]:
hnn

ANN_12_190827_1251    [70,  45,  30,  15]
ANN_24_190827_1325    [70,  45,  30,  15]
ANN_190827_1228       [60,  45,  30,  15]
Name: hidden_nodes, dtype: object

In [6]:
#viz = viz.copy()[['sequential_116_190819_040.h5', 'sequential_36_190818_1913.h5']]

In [18]:
sgd_cols = ['eta0', 'shuffle', 'fit_intercept', 'n_iter_no_change', 'penalty', 'max_iter', 'alpha', 'early_stopping',
           'learning_rate', 'loss', 'val_auc', 'test_auc']

rf_cols = ['n_estimators', 'max_depth', 'max_features', 'max_leaf_nodes', 
                                        'min_samples_leaf', 'min_samples_split', 'bootstrap',
                                        'criterion', 'val_auc', 'test_auc']

ann_cols = ['batch_size', 'optimizer', 'epochs_actual', 'class_1_weight', 'early_stopping', 'early_stopping_metric',
            'hidden_layers_no', 'hidden_nodes', 'hl_out_activations', 'loss_func', 'dropout', 'tr_accuracy',
             'val_auc', 'test_auc']

spider_cols = ['batch_size', 'epochs_actual', 'class_1_weight', 'hidden_layers_no',  'tr_accuracy', 'val_auc', 'test_auc']

ann_spider_cols_to_norm = ['epochs_actual', 'batch_size',  'tr_accuracy', 'val_auc', 'test_auc', 'class_1_weight', 'hidden_layers_no'] #'dropout',

In [19]:
sorted(list(viz.index))

['artifact_uri',
 'batch_and_steps',
 'batch_size',
 'bias_init',
 'class_1_weight',
 'dropout',
 'early_stopping',
 'early_stopping_metric',
 'end_time',
 'epochs_actual',
 'epochs_settings',
 'experiment_id',
 'experiment_type',
 'fn_rate',
 'fp_rate',
 'hidden_layers_no',
 'hidden_nodes',
 'hl_out_activations',
 'indexes_file_path',
 'kernel_init',
 'kernel_regularizers',
 'lifecycle_stage',
 'loss_func',
 'model_filename',
 'model_filepath',
 'model_type',
 'optimizer',
 'optimizer_settings',
 'pred_threshold',
 'roc_test_fpr',
 'roc_test_tpr',
 'roc_val_fpr',
 'roc_val_tpr',
 'run_id',
 'run_uuid',
 'start_time',
 'status',
 'test_auc',
 'test_file_path',
 'test_fn',
 'test_fnr',
 'test_fp',
 'test_fpr',
 'test_size',
 'test_tn',
 'test_tnr',
 'test_tp',
 'test_tpr',
 'tn_rate',
 'tp_rate',
 'tr_accuracy',
 'tr_auc_2',
 'tr_auc_3',
 'tr_auc_4',
 'tr_loss',
 'tr_precision_3',
 'tr_precision_4',
 'tr_precision_5',
 'tr_recall_3',
 'tr_recall_4',
 'tr_recall_5',
 'tr_time',
 'tr_time

In [20]:
colors = [TTQcolor['azureBlue'], TTQcolor['richOrange'], TTQcolor['algae'],
                                                       TTQcolor['yell'], TTQcolor['redBrown'], TTQcolor['bloodRed']]

In [21]:
#grid = performance_grid(viz, model_filter=None, single_row_folds=False, folds_p_width=600, single_spider=True, spider_p_width=600, spider_p_height=600,
#                       spreadsheet_cols = [])

In [22]:
viz.loc['model_type']

ANN_12_190827_1251    sequential
ANN_24_190827_1325    sequential
ANN_190827_1228       sequential
Name: model_type, dtype: object

In [23]:
spr_settings = {'viz_dict': viz, 
                                             'metric_list': ann_cols, 
                                             'model_type':  ['sequential'], 
                                             'color_cells':True, 
                                             'colors':colors,
                                             'index_header':'MLP', 
                                             'width':1200, 
                                             'height':120, 
                                             'index_width':40,
                                             'row_height':25}

In [24]:
grid = performance_grid(viz,
                     model_filter = None, 
                     legend_font_size='9pt', 
                     fpr_font_size='9pt', 
                     bestFprOnly=True, 
                     rocs_p_width=600, 
                     rocs_p_height=600,
                     rocs_line_width=2, 
                     single_spider=True, 
                     add_spider=True,
                     spider_in_row=2, 
                     spiders_params = spider_cols,
                     spider_p_width=600, 
                     spider_p_height=600, 
                     spider_text_size='12pt', 
                     spider_line_width=4.5, 
                     spider_fill_alpha=0.1,
                     spider_margin_distance=0.25, 
                     normalize_spider=True,
                        spiders_params_to_norm = ann_spider_cols_to_norm,
                     single_row_folds=True, 
                        add_corr_scatter=False,
                     folds_p_width=600, 
                     folds_p_height=600, 
                     folds_xlabelorientation=1.55, 
                     folds_group_text_font_size='3pt',
                     folds_in_row=2, 
                     spreadsheet_settings= [spr_settings],
                     #spreadsheet_cols = ann_cols,
                     #spr_index_header = 'MLP',
                     #spr_height=200, 
                     #spr_width=1600, 
                     #spr_index_width=25,
                     #spr_row_height=25,
                     plot_feat_importance=False,
                     normalize_importance=True,
                     fimp_text_group_size = '10pt',
                     colors=[TTQcolor['azureBlue'], TTQcolor['richOrange'], TTQcolor['algae'], TTQcolor['yell'], TTQcolor['redBrown'], TTQcolor['bloodRed']])

In [25]:
show(grid)

In [13]:
#from bokeh.io import export_png
#export_png(l, experiment+'.png')